In [1]:
import pickle

with open('bennyray_190916_18_bcn.pkl', 'rb') as f:
    connect_mats = pickle.load(f)

In [2]:
for subject in connect_mats:
    print(connect_mats[subject])

[[ 1.          0.26162214  0.02498825 ... -0.03269972  0.00112743
  -0.08957472]
 [ 0.26162214  1.          0.00603336 ... -0.01311578 -0.06174056
  -0.0031385 ]
 [ 0.02498825  0.00603336  1.         ... -0.0559236  -0.079857
   0.06500991]
 ...
 [-0.03269972 -0.01311578 -0.0559236  ...  1.          0.03162681
   0.0615443 ]
 [ 0.00112743 -0.06174056 -0.079857   ...  0.03162681  1.
   0.19448238]
 [-0.08957472 -0.0031385   0.06500991 ...  0.0615443   0.19448238
   1.        ]]
[[ 1.          0.17166853  0.07737673 ... -0.05626794  0.02805067
   0.11999321]
 [ 0.17166853  1.          0.0365655  ...  0.10917593  0.05712454
  -0.08847925]
 [ 0.07737673  0.0365655   1.         ...  0.00412536 -0.01782116
  -0.0784623 ]
 ...
 [-0.05626794  0.10917593  0.00412536 ...  1.         -0.07446056
   0.01893795]
 [ 0.02805067  0.05712454 -0.01782116 ... -0.07446056  1.
  -0.02812937]
 [ 0.11999321 -0.08847925 -0.0784623  ...  0.01893795 -0.02812937
   1.        ]]
[[ 1.          0.08903315  0.04763

In [3]:
import torch
from torch_geometric.data import Data

In [ ]:
# ==== from connectivity matrix to edges ====
edge_index_tmp = [[], []]
edge_attr_tmp = []

for idx, itm in enumerate(connect_mat):
    edge_index_tmp[0].extend([idx for i in range(len(itm)])
    edge_index_tmp[1].extend([i for i in range(len(itm)])
    for jdx, jtm in enumerate(itm):
        edge_attr_tmp.append([jtm])

edge_index = torch.tensor(edge_index_tmp, dtype=torch.long)
# where 0, 1 are the node indeces
# the shape of edge_index is [2, num_edges]

edge_attr = torch.tensor(edge_attr_tmp, dtype=torch.float)
# where the list items are the edge feature vectors
# the shape of edge_attr is [num_edges, num_edge_features]

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# where the list items are the node feature vectors
# the shape of x is [num_nodes, num_node_features]

y = torch.tensor([[0]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# ==== Create dataset with multiple data
from torch_geometric.data import DataLoader

data_list = [data, ..., data]
loader = DataLoader(data_list, batch_size=32, shuffle=true)

for batch in loader:
    pass # do operations on batches

import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}.format(acc)')
